# Amazon Bedrock Knowledge Bases

이 노트북은 실습을 위해 Amazon Bedrock Knowledge Bases를 생성하고, 데이터 소스에 해당하는 Amazon S3 버킷을 생성합니다.

#### 노트북 워크스루

현재 Git Repository의 dataset 디렉토리에 저장된 데이터를 Amazon S3 버킷에 업로드하고, AWS 콘솔에서 UI를 활용해 Amazon Bedrock Knowledge bases를 생성합니다. 

<!-- ![data_ingestion.png](./images/data_ingestion.png) -->
<img src="./images/data_ingestion.png" width=50% height=20% />

## 설정
나머지 셀을 실행하기 전에 아래 셀을 먼저 실행해 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

### Amazon S3에 데이터 업로드
벡터 데이터베이스에 벡터화 해서 저장할 문서 파일을 업로드 합니다. 

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
print(boto3.__version__)

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

data_bucket_name = f'bedrock-kb-{suffix}-1'
upload_directory("../dataset", data_bucket_name)

### Amazon Knowledge base 생성

1. AWS Console에서 Amazon Bedorck을 검색하여 Amazon Bedrock Console로 이동합니다.
![](images/rag_1.png)

2. 왼쪽 사이드바의 Knowledge bases를 선택한 후 Create 버튼의 [Knowledge Base with vector store]를 선택합니다. 
![](images/rag_2.png)

3. Knowledge Base name에 `bedrock-sample-knowledge-base`를 지정하고, IAM permission은 [Create and use a new service role]을 선택하여 필요한 기본 권한이 포함된 IAM Role을 새로 생성합니다. data source는 Amazon S3를 선택합니다.
![](images/rag_3.png)

4. Data source 설정에서 Data source name은 기본값을 그대로 사용합니다. 그리고 S3 URI 항목에서는 Browse S3 버튼을 클릭해서 앞서 문서를 업로드한 Amazon S3 Bucket을 선택합니다. (`bedrock-kb-`로 시작하는 bucket)
![](images/rag_4.png)

5. Parsing strategy는 기본값인 `Amazon Bedrock default parser`를 사용합니다. 이 Parser는 텍스트만 추출할 수 있기 때문에 이미지와 같은 멀티모달 컨텐츠들은 무시하고 가져옵니다.
![](images/rag_5.png)

6. Chunking strategy는 [Fixed-size chunking]을 선택해서 기본으로 지정된 최대 300 토큰수 이내로 청크를 나누도록 합니다. 이 때 청크의 20% 크기 정도는 overlap 되도록 설정합니다.
![](images/rag_6.png)


### Knowledge Base에 적재할 데이터 다운로드
다음 데이터를 사용합니다:
 - 첫 번째 데이터 소스로 로컬 디렉터리에 저장된 합성 데이터

#### 데이터를 S3 버킷 데이터 소스로 업로드

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../dataset", data_bucket_name)

### Ingestion 작업 시작
KB와 데이터 소스를 생성한 뒤 각 데이터 소스에 대해 ingestion 작업을 시작할 수 있습니다.
Ingestion 작업 동안 KB는 데이터 소스에서 문서를 가져와 텍스트를 추출하고, 지정한 청킹 크기에 따라 분할하며, 각 청크의 임베딩을 생성해 이번에는 OSS 벡터 데이터베이스에 저장합니다.

참고: 현재는 한 번에 하나의 ingestion 작업만 실행할 수 있습니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### 2.2 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `AWS의 Security 서비스들에는 어떤 것들이 있습니까?`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다:
```
AWS의 보안 서비스에는 다음이 포함됩니다:

- Amazon Cognito - Amazon Detective - Amazon GuardDuty - Amazon Inspector - Amazon Macie - Amazon Security Lake - Amazon Verified Permissions - AWS Artifact - AWS Audit Manager - AWS Certificate Manager - AWS CloudHSM - AWS Directory Service - AWS Firewall Manager - AWS Identity and Access Management - AWS Key Management Service - AWS Network Firewall - AWS Resource Access Manager - AWS Secrets Manager - AWS Security Hub CSPM - AWS Shield - AWS IAM Identity Center - AWS WAF - AWS WAF 캡차 이러한 서비스는 네트워크 보안, 구성 관리, 액세스 제어 및 데이터 암호화를 포함한 다양한 보안 기능을 제공합니다.
```

In [ ]:
query = "AWS의 Security 서비스들에는 어떤 것들이 있습니까?"

In [ ]:
foundation_model = "us.amazon.nova-pro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}:{}:inference-profile/{}".format(region, account_id, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Retrieve and Generate API를 사용하면 최종 응답을 바로 받게 되므로, 응답에 사용된 다양한 소스를 확인할 수 없습니다. 이제 Retrieve API로 Knowledge Base의 소스 정보를 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "Amazon ElastiCache가 제공하는 메모리 캐싱 엔진들에 대해 설명해주세요."
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

### 정리
아래 섹션의 주석을 해제한 뒤 실행해 모든 리소스를 삭제하세요.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
# knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)